In [1]:
from mammo_dataset import INbreastDataset
import pandas as pd
from glob import glob
import os.path as osp
import xml.etree.ElementTree as ET
import cv2

In [2]:
inbreast_dir = '/Users/lpires/Developer/dl/INbreast'
mammo_kp_dir = '/Users/lpires/Developer/dl/mammo_kp'

In [3]:
# ds = INbreastDataset(inbreast_dir, None)
# ds.read_dicoms()
# ds.read_cases()
# cases_df = ds.cases_df.reset_index()
# cases_df.to_csv('cases_df.csv', index=False)

In [4]:
cases_df = pd.read_csv('cases_df.csv')
kp_df = pd.DataFrame(
    [[int(osp.splitext(osp.basename(fn))[0]), fn] for fn in glob(osp.join(mammo_kp_dir, 'annotations', '*.xml'))],
    columns=['case_id', 'fn']
)

In [5]:
join_df = cases_df.join(kp_df.set_index('case_id'), how='inner', on='case_id')
join_df['helper'] = join_df.apply(lambda r: '%s%s' % (r['view'], r['laterality']), axis =1)

In [6]:
assert len(join_df) == len(kp_df)

In [7]:
#
# check duplicates
#

In [8]:
pivot = pd.pivot_table(join_df, index=['patient_id'], columns=['helper'], values=['fn'], aggfunc='count')['fn']

In [9]:
pivot = pivot[~pivot['MLOL'].isnull() & ~pivot['MLOR'].isnull()]

In [10]:
pivot_ones = pivot[(pivot['MLOL'] == 1.0) & (pivot['MLOR'] == 1.0)]
pivot_others = pivot[(pivot['MLOL'] != 1.0) | (pivot['MLOR'] != 1.0)]

In [11]:
assert len(pivot) == len(pivot_ones)
assert len(pivot_others) == 0

In [12]:
#
# mount imagens and keypoints matching
#

In [13]:
filtered_df = join_df[join_df['patient_id'].isin(pivot_ones.index)][['case_id', 'laterality', 'patient_id', 'fn']]

In [14]:
assert len(filtered_df) == 2 * len(pivot_ones)

In [15]:
data = {}
for _, r in filtered_df.iterrows():
    patient_id = r['patient_id']
    laterality = r['laterality']
    fn = r['fn']
    # parse lines
    tree = ET.parse(fn)
    root = tree.getroot()
    line_e = root.find('./object/line')
    line = [int(e.text) for e in list(line_e)]
    # build dict
    case_id = int(osp.splitext(osp.basename(fn))[0])
    if patient_id in data:
        x = data[patient_id]
    else:
        x = [None, None, None, None]
        data[patient_id] = x
    if laterality == 'L':
        x[0] = line
        x[2] = case_id
    elif laterality == 'R':
        x[1] = line
        x[3] = case_id
mount_df = pd.DataFrame.from_dict(data, orient='index', columns=['Lline', 'Rline', 'Lcase_id', 'Rcase_id'])
mount_df.sort_index(inplace=True)
mount_df.reset_index().to_csv('mount_df.csv', index=False)

In [16]:
# if want to check one case for debugging
# mount_df = mount_df[mount_df['Lcase_id'] == 22678694]

In [17]:
#
# draw samples
#

In [18]:
for patient_id, r in mount_df.iterrows():
    l_line = r['Lline']
    r_line = r['Rline']
    l_case_id = r['Lcase_id']
    r_case_id = r['Rcase_id']
    
    l_img = cv2.imread(osp.join(mammo_kp_dir, '%d.png' % l_case_id))
    r_img = cv2.imread(osp.join(mammo_kp_dir, '%d.png' % r_case_id))

    l_mask = cv2.imread(osp.join(mammo_kp_dir, 'annotations', '%d_pixels1.png' % l_case_id))
    r_mask = cv2.imread(osp.join(mammo_kp_dir, 'annotations', '%d_pixels1.png' % r_case_id))
    l_mask = cv2.bitwise_and(l_mask, (0, 255, 0), mask=cv2.cvtColor(l_mask, cv2.COLOR_BGR2GRAY))
    r_mask = cv2.bitwise_and(r_mask, (0, 255, 0), mask=cv2.cvtColor(r_mask, cv2.COLOR_BGR2GRAY))
    
    l_final = cv2.addWeighted(l_img, 1.0, l_mask, 0.05, 0)
    r_final = cv2.addWeighted(r_img, 1.0, r_mask, 0.05, 0)
    
    l_final = cv2.line(l_final, (l_line[0], l_line[1]), (l_line[2], l_line[3]), color=(0, 0, 255), thickness=5)
    r_final = cv2.line(r_final, (r_line[0], r_line[1]), (r_line[2], r_line[3]), color=(0, 0, 255), thickness=5)

    img_final = cv2.hconcat([r_final, l_final])
    
    cv2.imwrite(osp.join(mammo_kp_dir, 'qa', '%s.png' % patient_id), img_final)

In [19]:
mount_df[['Rcase_id', 'Lcase_id']].to_clipboard()

In [20]:
mount_df

,Lline,Rline,Lcase_id,Rcase_id
024ee3569b2605dc,"[371, 334, 24, 173]","[273, 355, 612, 174]",20588072,20588046
036aff49b8ac84f0,"[431, 423, 81, 241]","[395, 370, 764, 203]",20588680,20588654
069212ec65a94339,"[281, 316, 49, 255]","[414, 305, 596, 257]",50994733,50994760
0b7396cdccacca82,"[343, 383, 8, 193]","[466, 381, 754, 200]",22670878,22670855
1e10aef17c9fe149,"[425, 423, 49, 183]","[400, 414, 772, 181]",24055274,24055328
1f139436acfc5467,"[336, 454, 34, 230]","[477, 473, 761, 229]",50998413,50998440
21e6cc12630e5e9f,"[244, 314, 32, 226]","[369, 304, 630, 221]",53581860,53581887
2a5b932da4ce5ca1,"[344, 304, 85, 174]","[346, 327, 566, 181]",22580576,22580548
2dec4948fbe6336d,"[394, 336, 45, 165]","[278, 326, 630, 154]",22614568,22614545
2f1104b3cda7f145,"[255, 321, 3, 218]","[425, 252, 663, 125]",24054997,24055024
